# Building an Image Classifier Deep Network applying Transfer Learning

In [ ]:
import os

mount_path = os.environ['working_dir']

In [16]:
# import the required libraries
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# from tensorflow.keras.preprocessing import image_dataset_from_directory
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tf.keras.utils
from PIL import Image
import numpy as np
import os
import base64
from io import BytesIO

## Dataset creation

In [17]:
# dataset's directory path
# directory = "./images/"
directory = mount_path+"images/"

batch_size = 32
image_size = (160,160) # resize the images to 160x160

# create training and validation sets
# use image_dataset_from_directory to load the images
# set a validation split and specify the subset ('training' or 'validation')
# set the random seeds to match eachother (to avoid overlapping of the images in
# train and validation sets)

train_set = tf.keras.utils.image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=image_size,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42)
validation_set = tf.keras.utils.image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=image_size,
                                             validation_split=0.2,
                                             subset='validation',
                                             seed=42)

Found 183 files belonging to 2 classes.
Using 147 files for training.
Found 183 files belonging to 2 classes.
Using 36 files for validation.


# Obtain Class names (labels)

In [18]:
# print some images
# use .class_names attribute to retrieve the classes of the images from the dicrectories names
class_names = train_set.class_names
print(class_names)

['tea-earl-grey', 'tea-lemon']


# Import pre-trained model

In [19]:
# load the pre-trained model without the final layers
image_shape = image_size + (3,)
pre_trained_model = tf.keras.applications.MobileNetV2(input_shape=image_shape,
                                               include_top=False,
                                               weights='imagenet')

In [20]:
len(pre_trained_model.layers)

154

# Preprocessing layer

# Compile Model

# Train Model

# Plot Loss/Accuracy

# Verify Tea Lemon prediction results

# Verify Tea Earl-Grey prediction results

# With Data Augmentation

In [21]:
# freeze the base model by making it non trainable
pre_trained_model.trainable = False
# define the input layer
inputs = tf.keras.Input(shape=image_shape) 
# apply data augmentation
x = tf.keras.Sequential([tf.keras.layers.RandomFlip('horizontal'),
                         tf.keras.layers.RandomRotation(0.2)])(inputs)

# preprocess the augmented inputs
x = preprocess_input(x)
# add the pre-trained (not trainable) model
x = pre_trained_model(x, training=False)
# add a pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# add a dropout layer for regularization
x = tf.keras.layers.Dropout(0.2)(x)
# add the output layer
# outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
# outputs = tf.keras.layers.Dense(2, activation='sigmoid')(x)
# outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='sigmoid')(x)
# define the model with its inputs and outputs
model_augmented = tf.keras.Model(inputs, outputs)

In [22]:
model_augmented.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 160, 160, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 160, 160, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                           

# Compile augmented model

In [23]:
# compile the model
base_learning_rate = 0.001
model_augmented.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              # loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# train the model
history_augmented = model_augmented.fit(train_set, validation_data=validation_set, epochs=20)

Epoch 1/20


5/5 [==============================] - 7s 458ms/step - loss: 0.6819 - accuracy: 0.6190 - val_loss: 0.5371 - val_accuracy: 0.7222
Epoch 2/20
5/5 [==============================] - 3s 290ms/step - loss: 0.4947 - accuracy: 0.8095 - val_loss: 0.3675 - val_accuracy: 0.8056
Epoch 3/20
5/5 [==============================] - 3s 289ms/step - loss: 0.4289 - accuracy: 0.8163 - val_loss: 0.2939 - val_accuracy: 0.8889
Epoch 4/20
5/5 [==============================] - 3s 287ms/step - loss: 0.2863 - accuracy: 0.8639 - val_loss: 0.2629 - val_accuracy: 0.8889
Epoch 5/20
5/5 [==============================] - 3s 299ms/step - loss: 0.2113 - accuracy: 0.9116 - val_loss: 0.2561 - val_accuracy: 0.8889
Epoch 6/20
5/5 [==============================] - 3s 304ms/step - loss: 0.2088 - accuracy: 0.9184 - val_loss: 0.2461 - val_accuracy: 0.8889
Epoch 7/20
5/5 [==============================] - 3s 302ms/step - loss: 0.1701 - accuracy: 0.9456 - val_loss: 0.2320 - val_accuracy: 0.8889
Epoch 8/20
5/5 [===============

# test single image

In [24]:
# testfile = "./images/tea-earl-grey/PXL_20230803_124233750.MP.jpg"
# testfile = "./images/tea-lemon/PXL_20230803_153317298.jpg"

testfile = mount_path+"images/tea-earl-grey/PXL_20230803_124233750.MP.jpg"
# testfile = mount_path+"images/tea-lemon/PXL_20230803_153317298.jpg"

img = Image.open(testfile).convert('RGB')
img = img.resize((160, 160), Image.LANCZOS)
img = np.asarray(img)
# print(img)

mydata = np.empty((1, 160, 160, 3))
mydata[0] = img

# print(mydata)

mypredition = model_augmented.predict(mydata)

print(mypredition)
print(np.argmax(mypredition, axis=1))

1/1 [==============================] - 1s 1s/step
[[0.17160852 0.49986148]]
[1]


# Save augmented model

In [25]:
tf.saved_model.save(model_augmented, "./models/tea_model/1")

INFO:tensorflow:Assets written to: ./models/tea_model/1/assets


INFO:tensorflow:Assets written to: ./models/tea_model/1/assets


# Infer saved augmented model

### Load model

### Obtain In/Out keys

### Load image and predict

# Save model with Base64 Signature

### Load labels

In [26]:
print(class_names)
labels = tf.constant([class_names])
print(labels)

['tea-earl-grey', 'tea-lemon']
tf.Tensor([[b'tea-earl-grey' b'tea-lemon']], shape=(1, 2), dtype=string)


### Load augmented model

In [27]:
smodel = tf.saved_model.load("./models/tea_model/1")
# smodel = model_augmented

# This is the current signature, that only accepts image tensors as input
signature = smodel.signatures["serving_default"]
print(signature)

# obtain the key name of the output (typically 'dense_X')
keyOutput = next(iter(signature.structured_outputs))

@tf.function()
def my_predict(image_b64):

    # get content
    content = image_b64[0]
    
    # decode image    
    image = tf.image.decode_jpeg(content,channels=3)
    # tf.compat.v1.enable_eager_execution()
    
    # resize image
    image = tf.image.resize(image, [160, 160])
    
    # expand dimension to match signature
    image = tf.expand_dims(image, 0)
    
    # execute prediction
    prediction = signature(image)

    # obtain index of maximum probability prediction
    idx = tf.argmax(prediction[keyOutput],axis=1)
    
    # obtain the label for given index
    label = tf.gather(labels, idx, batch_dims=1)

    # obtain probability from Tensor
    probability = prediction[keyOutput][0,idx[0]]
    
    # combine result in String Tensor format with [label,probability]
    result = tf.concat([label, [tf.as_string(probability)]], axis=0)
    
    # return result_tensor
    return result

# Create new signature, to read b64 images
new_signature = my_predict.get_concrete_function(
    image_b64=tf.TensorSpec(name="image_b64", shape=[1], dtype=tf.string)
)

# Save model with Base64 input signature
tf.saved_model.save(
    smodel,
    # export_dir="./models/tea_model_b64/1",
    export_dir=mount_path+"models/tea_model_b64/1",
    signatures=new_signature
)

ConcreteFunction signature_wrapper(*, input_4)
  Args:
    input_4: float32 Tensor, shape=(None, 160, 160, 3)
  Returns:
    {'dense_1': <1>}
      <1>: float32 Tensor, shape=(None, 2)


INFO:tensorflow:Assets written to: ./models/tea_model_b64/1/assets


INFO:tensorflow:Assets written to: ./models/tea_model_b64/1/assets


# Test Base64 Model with single image

In [28]:
# smodel = tf.saved_model.load("../models/tea_model_b64/1")
smodel = tf.saved_model.load(mount_path+"models/tea_model_b64/1")


# Load model's signature
signature = smodel.signatures["serving_default"]

print(signature)

ConcreteFunction signature_wrapper(*, image_b64)
  Args:
    image_b64: string Tensor, shape=(1,)
  Returns:
    {'output_0': <1>}
      <1>: string Tensor, shape=(2,)


### Run Inference

In [29]:
# testfile = "./images/tea-earl-grey/PXL_20230803_124233750.MP.jpg"
# testfile = "./images/tea-lemon/PXL_20230803_153317298.jpg"

testfile = mount_path+"images/tea-earl-grey/PXL_20230803_124233750.MP.jpg"
# testfile = mount_path+"images/tea-lemon/PXL_20230803_153317298.jpg"


# load test image
content = tf.io.read_file(testfile)

# reshape to signature's expected dimensions
content = tf.reshape(content, shape = [1])

# print(tf.print(content, summarize=3))

# obtain signature
f = smodel.signatures["serving_default"]

# run prediction
myprediction = f(image_b64=content)
print(myprediction)

{'output_0': <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'tea-earl-grey', b'0.968096'], dtype=object)>}
